<a href="https://colab.research.google.com/github/techasit239/DADS5001-SUN/blob/main/HW2_Redesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

url = "https://fbref.com/en/comps/9/Premier-League-Stats"

tables = pd.read_html(url)

print(f"พบทั้งหมด {len(tables)} ตาราง")

df = tables[0].copy()

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [' '.join(col).strip() for col in df.columns.values]

df.columns = [c.replace(' ', '_').replace('%', 'Pct').replace('/', '_') for c in df.columns]

print(df.head())

df.to_csv("premier_league_2025.csv", index=False)


พบทั้งหมด 24 ตาราง
   Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts_MP    xG   xGA  xGD  \
0   1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2  9.3   
1   2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1 -0.1   
2   3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7 -0.8   
3   4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4 -1.8   
4   5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0  6.6   

   xGD_90     Last_5  Attendance                    Top_Team_Scorer  \
0    1.04  D W W W W       60144                Viktor Gyökeres - 3   
1   -0.01  D D W D W       11164                Antoine Semenyo - 6   
2   -0.09  D D W L W       61039  Richarlison, Micky van de Ven - 3   
3   -0.20  D W L W W       46335                  Wilson Isidor - 4   
4    0.74  D W W W L       52311                Erling Haaland - 11   

             Goalkeeper  Notes  
0            David Raya    NaN  
1

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

df = pd.read_csv("premier_league_2025.csv")
top10 = df.sort_values("Pts", ascending=False).head(10)
top20 = df.sort_values("Rk", ascending=False).head(20)
top6  = df.sort_values("Pts_MP",  ascending=False).head(6)

fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[0.5, 0.5],
    specs=[[{"type":"xy","colspan": 2}, None],
           [{"type":"table"},   {"type":"xy"}]],
    subplot_titles=(
        "GF vs Pts (bubble = Wins)",
        "Top-10 by Points สามารถเลื่อนคอลัมได้",
        "Standings (Top-20)",
         ),
    horizontal_spacing=0.08, vertical_spacing=0.12
)

# (1) Scatter: X=Pts, Y=GF, bubble=W
fig.add_trace(
    go.Scatter(
        x=df["Pts"], y=df["GF"],
        mode="markers+text",
        text=df["Squad"], textposition="top center",
        marker=dict(
            size=df["W"]*10, color=df["GF"],
            colorscale="Plasma", showscale=True, colorbar=dict(title="GF")
        ),
        hovertemplate="<b>%{text}</b><br>Pts %{x} · GF %{y} · W %{marker.size:.0f}<extra></extra>"
    ),
    row=1, col=1
)


# (2) Table: Top-10 Standings
top10 = top10.sort_values("Rk", ascending=True)
cols = ["Rk","Squad","MP","W","D","L","GF","GA","GD","Pts"]
fig.add_trace(
    go.Table(
        columnwidth=[10, 50, 20, 20, 20, 20, 20, 20, 20, 20],
        header=dict(values=[f"<b>{c}</b>" for c in cols],
                    fill_color="#2E2E3A", font=dict(color="white"), align="center"),
        cells=dict(values=[top10[c] for c in cols],
                   fill_color=[["#1f1f27","#222232"]*((len(top10)//2)+1)],
                   align="center")
    ),
    row=2, col=1
)

# (3) Bar: Top-10 Points
top10 = top10.sort_values("Pts")
fig.add_trace(
    go.Bar(
        x=top10["Pts"], y=top10["Squad"],
        orientation="h",
        marker=dict(color=top10["Pts"], colorscale="Plasma"),
        text=top10["Pts"], textposition="outside", showlegend=False
    ),
    row=2, col=2
)


# Layout
fig.update_xaxes(title="Pts (points win=3, draw=1)", row=1, col=1)
fig.update_yaxes(title="GF (Goals For)",  row=1, col=1)
fig.update_xaxes(title="Pts", row=1, col=2)
fig.update_layout(template="plotly_dark", height=850,
                  margin=dict(t=80,l=40,r=40,b=40))
fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv("premier_league_2025.csv")

cols = ["Rk", "Squad", "MP", "W", "D", "L", "GF", "GA", "GD", "Pts"]
df_table = df[cols]

fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=[f"<b>{c}</b>" for c in cols],
            fill_color="#003366",
            align="center",
            font=dict(color="white", size=13),
            height=30
        ),
        cells=dict(
            values=[df_table[c] for c in cols],
            fill_color=[["#f9f9f9", "#e3eaf2"] * (len(df_table)//2 + 1)],
            align="center",
            font=dict(color="#333333", size=12),
            height=28
        )
    )]
)

fig.update_layout(
    title="<b>Premier League 2024–25 Standings</b>",
    margin=dict(l=10, r=10, t=60, b=10),
    template="plotly_white"
)

fig.show()


In [ ]:
import plotly.express as px

fig = px.bar(
    df.sort_values("Pts", ascending=True),
    x="Pts",
    y="Squad",
    orientation="h",
    color="Pts",
    title="Premier League Standings 2025-2026",
    text="Pts"
)
fig.update_layout(template="plotly_dark", xaxis_title="Points", yaxis_title="Team")
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

fig = px.scatter(
    df,
    x="Pts", y="GD",
    size="W", color="W",
    hover_name="Squad",
    text="Squad",
    color_continuous_scale="Plasma",
    size_max=40,
    title="Premier League 2025–26: Goals vs Points (bubble = Wins)"
)


fig.update_traces(textposition="top center")
fig.update_layout(template="plotly_dark", margin=dict(t=80, l=30, r=30, b=30))
fig.show()
